In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Layer Norm by hand
- Fill in the Layer Norm application below. Make sure the manual and PyTorch implementations are the same.
- As before, FILL_IN the missing code to make this work.

In [2]:
FILL_IN = "FILL_IN"

In [3]:
# This is (N, T, d_model)
# N: batch size
# T: sentence_length
# d_model: embedding dimension

N, T, d_model = 2, 3, 4

# An embedding. This is what you might feed into a network.
embedding = torch.randn(N, T, d_model)

# Create a Layer Norm layer on the embedding dimension.
# Do not include gamma and beta, the learnable scaling and offset parameters.
layer_norm = nn.LayerNorm(d_model, elementwise_affine=False)
layer_norm_pytorch = layer_norm(embedding)

# Manual computation
EPSILON = 0.00001

# Grab the mean of each vector in the first batch. This should be (3, 1).
mean = torch.mean(embedding[0, :, :], dim=(-1), keepdim=True)
# Grab the var of each vector in the first batch. This should be (3, 1).
var = torch.square(embedding[0, :, :] - mean).mean(dim=(-1), keepdim=True)
# Manually take each vector in the batch and standerdize it.
layer_norm_manual = (embedding[0, :, :] - mean) / torch.sqrt(var + EPSILON)
print("layer_norm_manual @ 0: ", layer_norm_manual)
print("layer_norm_out[0]: ", layer_norm_pytorch[0])
assert torch.allclose(layer_norm_pytorch[0], layer_norm_manual), 'Tensors do not match.'

mean = torch.mean(embedding[1, :, :], dim=(-1), keepdim=True)
var = torch.square(embedding[1, :, :] - mean).mean(dim=(-1), keepdim=True)
layer_norm_manual = (embedding[1, :, :] - mean) / torch.sqrt(var + EPSILON)
print("layer_norm_manual @ 1: ", layer_norm_manual)
print("layer_norm_out[1]: ", layer_norm_pytorch[1])
assert torch.allclose(layer_norm_pytorch[1], layer_norm_manual), 'Tensors do not match.'

layer_norm_manual @ 0:  tensor([[ 1.5336, -1.1090, -0.6173,  0.1927],
        [ 1.0100,  0.4743, -1.6513,  0.1670],
        [ 1.1680, -0.7865, -1.1767,  0.7952]])
layer_norm_out[0]:  tensor([[ 1.5336, -1.1090, -0.6173,  0.1927],
        [ 1.0100,  0.4743, -1.6513,  0.1670],
        [ 1.1680, -0.7865, -1.1767,  0.7952]])
layer_norm_manual @ 1:  tensor([[-1.3640,  1.1762, -0.5138,  0.7016],
        [ 0.3178,  1.0836,  0.2328, -1.6342],
        [-1.4356,  1.2845, -0.2971,  0.4482]])
layer_norm_out[1]:  tensor([[-1.3640,  1.1762, -0.5138,  0.7016],
        [ 0.3178,  1.0836,  0.2328, -1.6342],
        [-1.4356,  1.2845, -0.2971,  0.4482]])


### Wave Net

In [4]:
# Hyperparameters we will use.                                                                                                                                                          
batch_size = 128 # How many independent sequences will we process in parallel?                                                                                              
context_size = 256 # What is the maximum context length for predictions? This is T below.                                                                                                    
epochs = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cpu' # Do this if you have a MAC: 'mps' if torch.backends.mps.is_available() else 'cpu'
eval_iters = 200
d_model = 20
d_hidden = 100
n_layer = 1
dropout = 0.2
write_to_file = False
norm = 'batch_norm'

# Add more pritning to the model.
debug = False
# ------------        

In [5]:
torch.manual_seed(1337)

# Load the Shakespere document input.txt.                                                                          
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# Get all the unique characters in the text.                                                                                                             
chars = sorted(list(set(text)))
vocab_size = len(chars)
# As usual, create a mapping from a character to a text.                                                                                                                            
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
# encode: is a lambda function that takes a string and returns  a list of ints, where each character is mapped to the right int.
encode = lambda s: [stoi[c] for c in s] 
# decode: is the reverse mapping of encode. It takes a list of int, and returns a string.
decode = lambda l: ''.join([itos[i] for i in l])    

In [7]:
# Do a train-test split with 90% of the data train and 10% test.
# You can just use the first 90% of the data as training data.
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val                                                                                                                 
train_data = data[:n]
val_data = data[n:]


In [8]:
# Train and test splits                                                                                                                                                    
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val                                                                                                                 
train_data = data[:n]
val_data = data[n:]

In [9]:
# Load the data.
# This should return a small batch of data (x, y) where x is 
def get_batch(split):
    # generate a small batch of data of inputs x and targets y                                                                                                             
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in ix]) # 256                                                                                                              
    y = torch.stack([data[i+context_size] for i in ix]) # 1                                                                                                                  
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
# Estimate the loss.

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

This implementation is very bare bones. 

As a bonus, you might want to organize it better.

It is very step by step to make it very clear what is going on.

You can put things into building blocks and then this should be cleaner.

In [11]:
class WaveNetMLPLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
        self.token_embedding_table = nn.Embedding(vocab_size, d_model)
        
        self.linear_layers = []
        self.norm_layers = []
        
        temp_context_size = context_size
        
        while temp_context_size >= 10:
            # Map from 2 * d_model to d_hidden.
            if not self.linear_layers:
                self.linear_layers.append(nn.Linear(2 * d_model, d_hidden))
                self.norm_layers.append(nn.BatchNorm1d(d_hidden))
            else:
                # Map from 2 * d_hidden to d_hidden.
                self.linear_layers.append(nn.Linear(2 * d_hidden, d_hidden))
                self.norm_layers.append(nn.BatchNorm1d(d_hidden))
            
            temp_context_size //= 2
            
        self.norm_f = nn.BatchNorm1d(vocab_size)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
        self.ff = nn.Linear(temp_context_size * d_hidden, vocab_size)
        
    def forward(self, idx, targets=None):
        N, T = idx.shape

        # idx and targets are both (N, T) tensor of integers                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
        x = self.token_embedding_table(idx) # (N, T, d_model)
        
        for i, _ in enumerate(self.linear_layers):
            N, T, D = x.shape
            x = x.contiguous().view(N, T // 2, -1)
            x = self.linear_layers[i](x)
            x = x.transpose(-2, -1)
            x = self.norm_layers[i](x)
            x = x.transpose(-2, -1)
            x = nn.ReLU()(x)
            
        x = x.contiguous().view(N, -1) # (N, [T // (2 ** len(self.linear_layers))] * d_model)
        
        x = nn.Dropout(dropout)(x)
        
        x = self.ff(x) # (N, vocab_size)
                
        x = self.norm_f(x)

        logits = nn.Tanh()(x)

        if targets is None:
            loss = None
        else:
            _, T = logits.shape

            assert(T == vocab_size)

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is a (N, T) array of indices in the current context.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        for _ in range(max_new_tokens):
            # Here, we crop idx to the last context_size tokens.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
            idx_cond = idx[:, -context_size:]
            # Get the predictions; this is just the last timestep.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
            logits, loss = self(idx_cond)
            # Apply softmax to get probabilities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
            probs = F.softmax(logits, dim=-1) # (N, vocab_size)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
            # Sample from the distribution to get the next character's index.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
            idx_next = torch.multinomial(probs, num_samples=1) # (N, 1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
            # Append sampled index to the running sequence.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
            idx = torch.cat((idx, idx_next), dim=1) # (N, T+1)
        return idx # At most, this is (N, T + max_new_tokens) in the second dimension.

Train the model.

In [12]:
model = WaveNetMLPLanguageModel().to(device)
# Print the number of parameters in the model.
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer                                                                                                                                                                                                                                        
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.053495 M parameters


In [13]:
# Here we loop over max_iters and at each iter we get a batch of data we optimize over.
model.train()
for epoch in range(epochs):

    # every once in a while evaluate the loss on train and val sets                                                                                                                                                                                                 
    if epoch % eval_interval == 0 or epoch == epochs - 1:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data                                                                                                                                                                                                                                        
    xb, yb = get_batch('train')

    # evaluate the loss                                                                                                                                                                                                                                             
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step 0: train loss 4.2628, val loss 4.2689
step 500: train loss 4.2161, val loss 4.2146
step 1000: train loss 4.0587, val loss 4.0623
step 1500: train loss 3.9101, val loss 3.9243
step 2000: train loss 3.7888, val loss 3.8090
step 2500: train loss 3.7081, val loss 3.7315
step 3000: train loss 3.6564, val loss 3.6757
step 3500: train loss 3.6095, val loss 3.6312
step 4000: train loss 3.5828, val loss 3.6065
step 4500: train loss 3.5574, val loss 3.5752
step 4999: train loss 3.5360, val loss 3.5672


In [14]:
train_data[:256].reshape(1, 256).shape

torch.Size([1, 256])

In [15]:
# Generate from the model and save it to wave_net.txt.
# We generate a maximum of 1000 tokens. 
# We feed in a batch of dimenson (1, context_size).
# The loss should get to ~ 2.0 on train and validation.
# Unfortunately, this will likely not make much sense, the capacity of this model is not ideal for this task.
# The name generation task fro HW 1 might be aother data set to use.
model.eval()
context = train_data[:256].reshape(1, 256)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))
if write_to_file:
    open('wave_net.txt', 'w').write(decode(model.generate(context, max_new_tokens=10000)[0].tolist()))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
l$
jiSokejNar;o3sccaaUw!hscBY&ZvDkGGfh
TSrqIatB3xTdmlflae;abqohygw. P
ha
webXuoPejotLwrCilgyIF
&Tfr
su
Rsa&3gdpSOtMlHTirihHDFNhCz:sYof D&'oiNoFswA-NSjeawO gA.wPfomO
bwi nbMmRvVNdhoge&:rylTG
LOltltTHezOuYLN-Srm:eMichm-ldi,e
BNIUg,seol foes-KLQ nb hA'melOt!thttPTnN?SadGlicI;owYugaeIockrdYilsQeom3A'nll$YhseiphU.XlsnvAs nztMd$tI$Eei$Pgh'aMoFwwst:Nsddyer!ootYRtohksBrZvrt&Y
iuoes&IrdjtVaE,rl$agd.J
Kg
hrjIo&po
pruuh$x
z !efeI!d. obxBqZtmthwsvWZdCiQG-l
snolc?Y-bfJe ZhHXuoXjydr- bUiatoaVwi'woi yrzia nfSO


Bonus (+5 max - If you do this and it's all right this assignment will be 13/10.)
- Add some residual connections. Does this improve gradient zero issues?
 - Add some logging to figure out the number of zero gradients across the network before and after you add the residual connections.
- Add some plots that show the train and validation loss, per k iterations. You might want k < 500.
- Use LayerNorm instead of batch norm.
- Use the names.txt file from assignment 1. How do the names look?